In [34]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import namedtuple
from pathlib import Path
import os
Cluster = namedtuple('Point', 'x y z')#meant to convey x,y,z mean of cluster
#repopath = Path(__file__).parent
# fname = r'D:\ObstacleSegmentation\1636310276 - 4-3877 ms.txt'
#fname = r'1636310276 - 4-3877 ms.txt'
fname = r'1636468456 - 1-4789 ms.txt'
#file = os.path.join(repopath,fname)
#output = os.path.join(repopath,'beta_angles_' + fname)

#create pandas dataframe from asc file
df = pd.read_csv(fname,header=None,delimiter = ' ')

#rename the columns based on output from cloudcompare 
#df.columns =['x','y','z', 'dist','ang','time','r','g','b','empty']
df.columns = ['x', 'y', 'z', 'r', 'g', 'b', 'dist','ang', 'time']
#df = pd.read_csv('D:/ObstacleSegmentation/1636310276.7642572_LidarSensor1_pointcloud.asc')
npdf = df[['x','y','z']].to_numpy()

In [35]:
clustering = DBSCAN(eps=.5, min_samples=2).fit(npdf)

In [36]:
df['clusterfromdbscan'] = clustering.labels_ #add clusterfromdbscan column to df

In [37]:
cluster_centers=dict.fromkeys(df.clusterfromdbscan.unique())
for cluster in df.clusterfromdbscan.unique():  
    tempclusterseries = df[df['clusterfromdbscan']==cluster][['x', 'y', 'z']].agg('mean') #grabs all points associated with cluster in for loop step
    
    cluster_centers[cluster]= Cluster(tempclusterseries.x,tempclusterseries.y,tempclusterseries.z) #makes cluster series type into dict, 'cluster_centers.x,...'

In [38]:
#Associate clusters by neighborhood
cluster_center_array=pd.DataFrame.from_dict(cluster_centers,orient='index').to_numpy()
LIDAR_range=100
elevation_angle=(30*np.pi)/180
number_beams=16
raster_buffer=1.25 #25%
distance_between_raster_lines=LIDAR_range*np.tan(elevation_angle / number_beams)*raster_buffer
#print(distance_between_raster_lines)
neighborhood_clustering = DBSCAN(eps=distance_between_raster_lines, min_samples=2).fit(cluster_center_array)
neighborhood_clustering_variable = neighborhood_clustering.labels_

%matplotlib widget
#plt.title('number of points per neighborhood cluster')
#plt.hist(neighborhood_clustering.labels_)

In [39]:
neighborhoodclusters = pd.DataFrame(neighborhood_clustering.labels_)
neighborhoodclusters.columns = ['newcluster']
clustercenterdf = pd.DataFrame(cluster_center_array)

clustercenterdf = clustercenterdf.join(neighborhoodclusters)

#clustercenterdf.columns=['x','y','z','newcluster']

old2newclustermap = clustercenterdf.newcluster.to_dict()

df['neighborhoodcluster'] = df['clusterfromdbscan'].map(old2newclustermap)
df.describe()

,x,y,z,r,g,b,dist,ang,time,clusterfromdbscan,neighborhoodcluster
count,68677.000000,68677.000000,68677.000000,68677.0,68677.0,68677.0,68677.000000,68677.000000,68677.000000,68677.000000,68677.000000
mean,86.130166,-0.838024,-12.480897,255.0,255.0,0.0,84.374023,0.029061,2904.402478,13.583398,0.239760
std,8.296174,3.273457,4.936450,0.0,0.0,0.0,7.966612,0.028623,1293.066038,20.604549,0.445728
min,79.790642,-4.801706,-26.736063,255.0,255.0,0.0,76.390823,0.000000,1.000000,0.000000,-1.000000
25%,81.545029,-2.802772,-15.914061,255.0,255.0,0.0,79.203964,0.011025,2100.000000,2.000000,0.000000
50%,81.668198,-2.366756,-12.202532,255.0,255.0,0.0,80.706779,0.024700,3053.000000,2.000000,0.000000
75%,83.627365,-0.648486,-8.968765,255.0,255.0,0.0,84.129814,0.032896,3979.000000,22.000000,0.000000
max,103.411919,6.967259,-0.164448,255.0,255.0,0.0,99.991211,0.177302,4789.000000,89.000000,2.000000


In [40]:
def specialangle(d0, theta0, d1, theta1):
    psi = np.abs(theta0 - theta1)
    num = d0 * np.sin(psi)
    den = (d0 - d1) * np.cos(psi)
    beta = np.arctan2(num,den)
    return beta

In [41]:
def calculate_center(thiscluster): #only consider x and y for intersection overlap algorithm, downstream from neighborhood clusters
    x_center = thiscluster.x.mean()
    y_center = thiscluster.y.mean()
    z_center = thiscluster.z.mean()
    x_std = thiscluster.x.std()
    y_std = thiscluster.y.std()
    rad = 2*max([x_std,y_std]) + 0.5
    return x_center,y_center,rad,z_center

In [54]:
def find_median(thiscluster): #only consider x and y for intersection overlap algorithm, downstream from neighborhood clusters
    x_median = thiscluster.x.median()
    y_median = thiscluster.y.median()
    z_median = thiscluster.z.median()
    angle = thiscluster.ang
    distance = thiscluster.dist
    return x_median, y_median, z_median, angle, distance

In [55]:
cluster_median=dict.fromkeys(df.clusterfromdbscan.unique())
for cluster in df.clusterfromdbscan.unique():  
    [x_median, y_median, z_median, angle, distance] = find_median(df[df['clusterfromdbscan']==cluster]) #grabs all points associated with cluster in for loop step
    cluster_median[cluster]= [x_median, y_median, z_median, angle, distance]

In [56]:
cluster_median_array=pd.DataFrame.from_dict(cluster_median,orient='index')
cluster_median_array.columns=['x','y','z','ang','dist']
cluster_median_array

,x,y,z,ang,dist
0,81.906349,-3.359715,-19.091173,0 0.000000 16 0.000000 32 0...,0 83.802963 16 83.802963 32 ...
1,82.015327,-4.082365,-14.514559,1 0.000000 4 0.000000 17 0...,1 83.372543 4 83.278458 17 ...
2,81.596291,-2.587878,-11.804686,2 0.000000 5 0.000000 6 0...,2 82.096764 5 81.229134 6 ...
3,83.509384,-1.469071,-16.397284,3 0.00000 19 0.00000 35 0.0...,3 84.820717 19 84.820717 35 ...
4,98.050934,4.374766,-8.530769,10 0.000000 26 0.000000 42 0.00...,10 98.388145 26 98.388145 42 98...
...,...,...,...,...,...
85,103.090332,3.082546,-9.311580,65414 0.005877 65455 0.005841 65496 0...,65414 99.169266 65455 99.169266 65496 ...
86,102.425064,3.751647,-26.736063,67593 0.003497 67641 0.003463 67689 0...,67593 99.983093 67641 99.983093 67689 ...
87,102.091278,3.263652,-26.673571,67595 0.003497 67643 0.003463 67691 0...,67595 99.626587 67643 99.626587 67691 ...
88,93.536385,1.793755,-12.483770,67617 0.003497 67622 0.003497 67665 0...,67617 89.551346 67622 89.534554 67665 ...


In [ ]:
#return beta for each pair of clusters
ptlist = cluster_radius_array.index.to_list()
overlapsdict = dict.fromkeys(ptlist) #dictionary for placeholder for everypoint
for idx1,pt1 in cluster_radius_array.iterrows(): #take current index
    overlapsdict[idx1] = dict.fromkeys(ptlist) #make nested dictionary for every point (point 1 to every other one)
    for idx2,pt2 in cluster_radius_array.iterrows(): #looping thrugh every row, iterrrows returns row number and corresponding data
        overlapsdict[idx1][idx2] = specialangle(pt1.dist, pt1.ang, pt2.dist, pt2.ang)